In [ ]:
from __future__ import print_function
import os
import sys
sys.path.append(os.path.join(os.environ['ITHEMAL_HOME'], 'learning', 'pytorch'))

In [ ]:
%matplotlib inline

import numpy as np
import torch
import data.data_cost as dt
import common_libs.utilities as ut
import models.train as tr
import models.graph_models as md
import models.losses as ls
import random

In [ ]:
data = dt.load_dataset('/home/ithemal/ithemal/learning/pytorch/inputs/embeddings/code_delim.emb', data_savefile='/home/ithemal/ithemal/learning/pytorch/saved/time_skylake_1217.data')

ys = np.array([d.y for d in data.train])
mean = ys.mean()
std = ys.std()

In [ ]:
embedding_size = data.final_embeddings.shape[1]
model = md.GraphNN(embedding_size, 256, 1, False)
model.set_learnable_embedding(mode = 'none', dictsize = max(data.word2id) + 1, seed = data.final_embeddings)

In [ ]:
train = tr.Train(model, data, 4)
train.loss_fn = ls.mse_loss
train.print_fn = train.print_final
train.correct_fn = train.correct_regression
train.num_losses = 1

In [ ]:
state_dict = torch.load('../saved/edges_none_01-07-19_06:42:33.mdl')
model.load_state_dict(state_dict['model'])

In [ ]:
model

In [ ]:
def evaluate(item):
    item.block.remove_edges()
    for i in item.block.instrs:
        print(i)
    prediction = model(data.train[0]).item()
    actual = item.y
    print('P: {:.2f}, A: {:.2f}'.format(prediction, actual))

In [ ]:
for item in data.data:
    item.block.remove_edges()

In [ ]:
actual, predicted = train.validate('/tmp/foo')

In [ ]:
item = data.test[30]

model.remove_refs(item)
model.init_bblstm(item)
roots = item.block.find_roots()

root_hidden = []
for instr in roots:
    token_embeds_lstm = torch.FloatTensor(instr.tokens).unsqueeze(1)
    _, (ins_embed, _) = model.lstm_token(token_embeds_lstm, model.init_hidden())
    _, (ins_hidden, _) = model.lstm_ins(ins_embed, model.init_hidden())
    root_hidden.append(ins_hidden.squeeze())

final_hidden = root_hidden[0]
for hidden in root_hidden[1:]:
    final_hidden = model.reduction(final_hidden,hidden)
pred = model.linear(final_hidden).squeeze()

total_weight = model.linear.weight.data.squeeze().abs().sum()

for i in range(len(root_hidden)):
    max_vals, max_idxs = torch.stack(root_hidden).max(dim=0)
    idxs = np.where(max_idxs.data.numpy() == i)
    i_vals = max_vals[idxs]
    i_weights = model.linear.weight.data.squeeze()[idxs]
    i_w_contrib = (i_weights * i_vals).sum()
    print('{:<30}: weight {:.2f}, total contrib {:.2f}'.format(
        item.block.instrs[i],
        i_weights.abs().sum() / total_weight,
        i_w_contrib,
    ))

print('\npred: {:.2f}, actual: {:.2f}'.format(
    pred.item(),
    item.y,
))

In [ ]:
import itertools
from tqdm import tqdm

dataset = data.train

starts = [None] * len(dataset)
ends = [None] * len(dataset)

for i, datum in enumerate(dataset):
    starts[i] = datum.y * 0.75
    ends[i] = datum.y * 1.25

starts.sort()
ends.sort()

xs = []
ys = []

max_val = 0
max_count = 0
curr_count = 0
for val, typ in sorted(itertools.chain(zip(starts, itertools.repeat('start')), zip(ends, itertools.repeat('end')))):
    if typ == 'start':
        curr_count += 1
        if curr_count > max_count:
            max_count = curr_count
            max_val = val
    elif typ == 'end':
        curr_count -= 1
    xs.append(val)
    ys.append(curr_count)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(xs, ys)

In [ ]:
p_2 = np.ones_like(actual) * 33 + 1e-2

In [ ]:
np.sum(np.abs(actual - p_2) / (p_2 + 1e-3) * 100 < 25)